### Imports

In [1]:
from tools import *
import math
import warnings
warnings.filterwarnings("ignore")

### API

- Dados de Usuário

In [2]:
user = "190126230@aluno.unb.br"
password = "ENG1901me00$$"

### Dynamics CRM Data Retrieval Workflow

1. **Initialize WebDriver for Chrome:**


2. **Azure AD and OAuth2.0 Parameters:**


3. **Construct Authorization URL:**


4. **Retrieve Callback URL after Authentication:**


5. **Get Access Token:**


6. **Define Dynamics CRM API Endpoint:**


7. **Make Authenticated Request to API Endpoint:**


8. **Process Response Data:**



In [3]:
# Initialize WebDriver for Chrome
driver = webdriver.Chrome()

# Azure AD and OAuth2.0 parameters
url = 'https://org425ee2cf.api.crm.dynamics.com'
client_id = '51f81489-12ee-4a9e-aaae-a2591f45987d'
callback_base_url = 'https://localhost'

# Construct authorization URL
auth_url = (
    f'https://login.microsoftonline.com/common/oauth2/authorize?'
    f'client_id={client_id}&response_type=code&redirect_uri={callback_base_url}&resource={url}'
)

# Retrieve callback URL after authentication
callback_url = take_callback_url(auth_url, driver, user, password)

# Get access token
access_token = get_access_token(callback_url, client_id, callback_base_url, url)

# Define Dynamics CRM API endpoint
api_endpoint = 'https://org425ee2cf.crm.dynamics.com/api/data/v9.2/crddb_autorowreportinputsessions?$top=10'

# Make authenticated request
data = make_authenticated_request(access_token, api_endpoint)

# Process the response
print(data)

# Create DataFrame from response data
data_df = pd.DataFrame(data['value'])
data_df.set_index("crddb_autorowreportinputsessionid", inplace=True)


https://localhost/?code=0.AQoAoZs17AtjK024M8jm1I-AWYkU-FHuEp5Kqq6iWR9FmH0BAAA.AgABAAIAAAAmoFfGtYxvRrNriQdPKIZ-AgDs_wUA9P-Nx97JSRBdK6A5uFLW69A6wxbaboJ5oVRZMeFPdn1oiwyXOXsmuB9Wb7Mh9TvvrbxzkrMapyd-tG4OAcIAwQHta8ZwpX_Q83qNpbMcbfq7dBRUgxuhmY_YXQ8uLpVwlOwplZgyEu26la7D-aw3khcbQ9P4KUoNTRjmW46yD2BZ0C6PcKCBm4ggTcSZevxwo5GKvHAUni3qC9WMry8KtkmDQS5WkkT_RqG29LmOU_NPGjVgrIka8Gfn13YuecG4XQR5kRY0NzdBnHcHV0jEY0G4G_0SrbCthRgA7oRTM4yaM2a_Vgd89Z-iwoMw9GilZGOg30CoUe-WMK5TKd34YCrOLNoWddV7Na7YYwYb15Qgw5IEToeDJFsZiinYGBNJtcfyH6j2npIseePVV-KCCgUnKwNy5sVIE_1dGTNWu3ahHvEuVCaSwOpADxYVVULGlUigaVcUo26GRrDbJNXOHgY95vxd8-TIGymrGq_ahLsiTs1BcQ0XBz3XsTwCzMovbHCJQyLNO-zIFhZ2K7rKHCK5CApir9DGW-5BjNtB_ZMGhwNz0iXEbuc1PQ4MbC4tlawuAplH6N5wuLlp9UYAozIrz-pDg2TBMq7j8YETBxizJsf2rhBg1JhQmRBT0X1S_7Y_T1bO4Ht3GG2Es4RXAmCtyWKKq72XCgJdVsc&session_state=9a3c4fc3-263a-4b95-a631-05dce250ecd9
eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IlQxU3QtZExUdnlXUmd4Ql82NzZ1OGtyWFMtSSIsImtpZCI6IlQxU3QtZExUdnlXUmd4Ql82NzZ1OGtyWFMtSSJ9.eyJhdWQiOiJodHRwcz

In [4]:
#  Chame a função e passe o caminho do seu dataset como argumento
check_equal_columns(data_df)

O dataset não possui colunas com nomes iguais.


False

## Coordinate LT

In [5]:
id_projects = list(data_df.index)
name_target = list(data_df['crddb_coordinatesfile_target_name'])
name_source = list(data_df['crddb_coordinatesfile_source_name'])

for i, target, source in zip(id_projects, name_target, name_source):
    downloaded_file(i, 'target', target, access_token)
    downloaded_file(i, 'source', source, access_token)
        

File downloaded successfully.
File downloaded successfully.
Failed to download the file. Status code: 404
Failed to download the file. Status code: 404


In [6]:
lt_coor = pd.read_csv(name_source[0], header=None)

n_points_before = data_df['crddb_spansbeforeemizone'][0]  # Number of points to add before
n_points_after = data_df['crddb_spansafteremizone'][0]   # Number of points to add after
distance_between_points = data_df['crddb_averagespanlength'][0]  # Euclidean distance between points

lt_coor_ = create_orientation_points(
    lt_coor, n_points_before, n_points_after, distance_between_points)

lt_coor_final = lt_coor_[[0,1]]
lt_coor_final = lt_coor_final.astype(float)
new_row = lt_coor_final.iloc[0].copy() + 0.01

# Insert the new row at index 0
lt_coor_final = pd.concat([pd.DataFrame(new_row).transpose(), lt_coor_final], ignore_index=True)

# Monitor Fault

In [7]:
from tools_fault import *

Parâmetros relativos ao tipo de circuito

In [8]:
circuit_type = data_df.iloc[0]['crddb_circuittype']  

victim_phase = 5 if circuit_type == 1 else 8
ref_phase = 4 if circuit_type == 1 else 7
ref_phase = 4 if circuit_type == 1 else 7

Contribution

In [9]:
contri_term1_mod = data_df['crddb_contributionfaultcurrent_term1_mod'][0]
contri_term2_mod = data_df['crddb_contributionfaultcurrent_term2_mod'][0]

contri_term1_phase = data_df['crddb_contributionfaultcurrent_term1_phase'][0]
contri_term2_phase = data_df['crddb_contributionfaultcurrent_term2_phase'][0]

print(f'Contribution from terminal 1: Magnitude {contri_term1_mod} Angle {contri_term1_phase}')
print(f'Contribution from terminal 1: Magnitude {contri_term2_mod} Angle {contri_term2_phase}')

Contribution from terminal 1: Magnitude 5000.0 Angle 0.0
Contribution from terminal 1: Magnitude 5000.0 Angle 0.0


Fault Location

In [10]:
faultLocation_Section = int(data_df["crddb_faultlocation_section"][0])
faultLocation_Section

15

In [11]:
length = calculate_length(lt_coor_final)
length = math.ceil(length)
print(f"Length between the start and end points of the DataFrame: {length}")

Length between the start and end points of the DataFrame: 3220


In [12]:
fault = f'''  MONITOR-FAULt
    REF-SPLITS-FILE,YES,.\Sp_SteadyState_0.f05
      FAULT-PHASE,1
      REF-PHASE,{ref_phase}
      CONNECTION-Impedance,0.0001,0
      SOURCE-TYPE,1
        SOURCE_CURRENT,Linear,
      LOCATION-TYPE,SECTION,1000
      TERMINAL,Term1
      TERMINAL,Term2
        LOCATION,{faultLocation_Section},{faultLocation_Section},1
      MONITOR-DATA,YES
        DATA,1,1,0,0,0,1,0
        VICTIM-PHASE,{victim_phase}

      CURRENT SOURCE
        SOURCE-TERM,1,2
        INTERPOLATED-PHASES,1
        INTERPLT-METHOD,BasedOnFaultedPhase
        KNOWNFAULT-Pts,2
          FAULT-LOCATION,0
            TERM_FAULTCURREN,{contri_term1_mod},{contri_term1_phase}
            TERM_FAULTCURREN,{contri_term1_phase},{contri_term1_phase}
          FAULT-LOCATION,{length}
            TERM_FAULTCURREN,{contri_term2_mod},{contri_term2_phase}
            TERM_FAULTCURREN,{contri_term2_phase},{contri_term2_phase}

TOTAL-INTERFerence
  FAULT-TYPE,ALL,0
  ADDITION-MALZ,,2
  INDUCTION,YES
  SUBDIVISION,NO,0
  SUBDIVISION,NO
  WORKTYPE,PHASE
  IMPED-CORRECT,NO
  PATHS-PROFILES
    WINDOW-PROFILES,NO,
    INCLUDED-PATHS-PROFILES,YES,5
    COATING-CORRECTION,YES,
      SURFACE-PROFile,POINTS,1
  TOWER-SELECTION,YES,
    LIMIT-IMPEDAnce,20
    LOCATION-PATh,4
    ROTATETOWER,YES
    EXPORT-TOWER-Ref,PATH
    GROUND-CONFIguration,ARRAY,3,5,0.00794
  ENERGIZATION-busses,CURRENT
    SELECTION-PRocess
      SHUNT-CURRENT,YES,1
      TOWER-LOCATIon,NO
      USE-TERMINAL-GRD,NO
    GLOBAL-OFFSET,NO,-1000,-1000

ENDPROGRAM'''

In [13]:
# File paths
input_file_path = 'C:/Codigos_VS/Projects/PowerApps/RW_SteadyState.f05'
output_file_path = input_file_path # 'C:/Codigos_VS/Projects/PowerApps/RW_SteadyState.f05'

# Text to search for and replacement text
search_text = "MONITOR-FAULt"
replacement_text = fault  # Replace with your desired text

# Call the function to modify the file content and save to a new file
modify_file_and_save(input_file_path, output_file_path, search_text, replacement_text)


File 'C:/Codigos_VS/Projects/PowerApps/RW_SteadyState.f05' created with the modification.
